In [1]:
import numpy as np
import pandas as pd
import uuid
import allel


absp /home/gridsan/djuna/.conda/envs/wgs_env/lib/python3.10/site-packages/tabixpy


In [2]:
path = './test_output/DEJ_11898_B01_GRM_WGS_2017-05-15_19.recalibrated_variants.annotated.vcf.gz'
path2 = './test_output/DEJ_11898_B01_GRM_WGS_2017-05-15_19.recalibrated_variants.annotated.vcf.gz.tbi'

In [4]:
dictionary = np.load('./human_Release_19_GRCh37p13/pos_dictionary.npy', allow_pickle=True).item()

In [181]:
d = pd.DataFrame(pd.read_table('./test_output/DEJ_11898_B01_GRM_WGS_2017-05-15_19.recalibrated_variants.annotated.coding.txt'))

/state/partition1/slurm_tmp/21190112.0.0/ipykernel_84177/1477470787.py:1: DtypeWarning: Columns (34,44,46,78,81) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.DataFrame(pd.read_table('./test_output/DEJ_11898_B01_GRM_WGS_2017-05-15_19.recalibrated_variants.annotated.coding.txt'))


In [150]:
f = open('log.txt', 'w')

path = './test_output/DEJ_11898_B01_GRM_WGS_2017-05-15_19.recalibrated_variants.vcf.gz'
path2 = './test_output/DEJ_11898_B01_GRM_WGS_2017-05-15_19.recalibrated_variants.vcf.gz.tbi'
callset = allel.read_vcf(path, log = f, fields = '*', region=dictionary['ABCA7'], tabix=path2) ### need to get the sample calls / annotations

f.close()

In [298]:
callset.keys()

dict_keys(['samples', 'calldata/AB', 'calldata/AD', 'calldata/DP', 'calldata/GQ', 'calldata/GT', 'calldata/MIN_DP', 'calldata/MQ0', 'calldata/PGT', 'calldata/PID', 'calldata/PL', 'calldata/RGQ', 'calldata/SB', 'variants/AC', 'variants/AF', 'variants/ALT', 'variants/AN', 'variants/BaseQRankSum', 'variants/CHROM', 'variants/DP', 'variants/DS', 'variants/END', 'variants/ExcessHet', 'variants/FILTER_LowQual', 'variants/FILTER_PASS', 'variants/FILTER_VQSRTrancheINDEL99.00to100.00', 'variants/FILTER_VQSRTrancheINDEL99.00to100.00+', 'variants/FILTER_VQSRTrancheSNP99.80to100.00', 'variants/FILTER_VQSRTrancheSNP99.80to100.00+', 'variants/FS', 'variants/HaplotypeScore', 'variants/ID', 'variants/InbreedingCoeff', 'variants/MLEAC', 'variants/MLEAF', 'variants/MQ', 'variants/MQ0', 'variants/MQRankSum', 'variants/NEGATIVE_TRAIN_SITE', 'variants/POS', 'variants/POSITIVE_TRAIN_SITE', 'variants/QD', 'variants/QUAL', 'variants/REF', 'variants/ReadPosRankSum', 'variants/SOR', 'variants/VQSLOD', 'variants

array([[4.181e-04,       nan,       nan],
       [4.181e-04,       nan,       nan],
       [5.017e-03,       nan,       nan],
       ...,
       [8.361e-04,       nan,       nan],
       [4.181e-04,       nan,       nan],
       [7.170e-01,       nan,       nan]], dtype=float32)

In [302]:
df1 = pd.DataFrame(np.concatenate((callset['variants/CHROM'].reshape(-1,1), callset['variants/POS'].reshape(-1,1), 
               callset['variants/REF'].reshape(-1,1), callset['variants/ALT'],
               callset['variants/FILTER_PASS'].reshape(-1,1), callset['variants/numalt'].reshape(-1,1), callset['variants/AF']), axis = 1))
df1.columns = ['CHROM', 'POS', 'REF_0', 'ALT_1', 'ALT_2', 'ALT_3', 'FILTER_PASS', 'NUMALT', 'AF_1', 'AF_2', 'AF_3']
df2 = (allel.GenotypeArray(callset['calldata/GT']))


In [303]:
df1

,CHROM,POS,REF_0,ALT_1,ALT_2,ALT_3,FILTER_PASS,NUMALT,AF_1,AF_2,AF_3
0,19,1040117,A,G,,,True,1,0.000418,NaN,NaN
1,19,1040272,C,T,,,True,1,0.000418,NaN,NaN
2,19,1040308,G,A,,,True,1,0.005017,NaN,NaN
3,19,1040382,G,A,,,True,1,0.046,NaN,NaN
4,19,1040390,C,T,,,True,1,0.000418,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
623,19,1065197,G,C,,,True,1,0.717,NaN,NaN
624,19,1065304,C,A,,,True,1,0.000418,NaN,NaN
625,19,1065305,G,A,,,True,1,0.000836,NaN,NaN
626,19,1065410,G,C,,,True,1,0.000418,NaN,NaN


In [ ]:
# merging on position: postion to index dictionary 

In [304]:
dd = pd.merge(df1, d[['POS', 'ID', 'REF', 'ALT', 'EFFECT', 'IMPACT', 'GENE', 'GENEID', 'HGVS_C', 'HGVC_P', 'LOF', 'NMD']], on = 'POS')

In [305]:
dd = dd.loc[dd['FILTER_PASS']]

In [306]:
pos = callset['variants/POS']
indices = dict(zip(pos, range(len(pos))))

In [307]:
select = [indices[i] for i in dd['POS']]

In [308]:
genotype = callset['calldata/GT'][select]

In [309]:
def convert_genotypes_to_str(genotype):
    converted = [str(i[0]) + '/' + str(i[1]) for i in genotype]
    return converted

In [310]:
genotypes_df = pd.DataFrame([convert_genotypes_to_str(x) for x in genotype])

In [311]:
genotypes_df.columns = callset['samples']

In [312]:
temp = np.unique(genotypes_df.iloc[0], return_counts = True)
temp2 = np.unique(genotypes_df.iloc[1], return_counts = True)


In [313]:
g = np.unique(genotypes_df)

frequencies = pd.DataFrame([[np.sum(genotypes_df.iloc[i]==x) for x in g] for i in range(genotypes_df.shape[0])])
frequencies.columns = 'N ' + g

In [314]:
all_variants = pd.concat((dd, frequencies, genotypes_df), axis = 1)

,CHROM,POS,REF_0,ALT_1,ALT_2,ALT_3,FILTER_PASS,NUMALT,AF_1,AF_2,...,SM-CTEN7,SM-CTEN8,SM-CTEN9,SM-CTENA,SM-CTENC,SM-CTEND,SM-CTENE,SM-CTENF,SM-CTENG,SM-CTENH
0,19,1041971,T,G,,,True,1,0.001672,NaN,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
1,19,1042158,G,C,,,True,1,0.000836,NaN,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,19,1042791,G,A,,,True,1,0.000418,NaN,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3,19,1042809,A,G,,,True,1,0.457,NaN,...,1/1,0/0,0/1,0/0,1/1,0/1,1/1,0/0,0/1,0/1
4,19,1043103,G,A,,,True,1,0.055,NaN,...,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,19,1065018,G,T,,,True,1,0.716,NaN,...,1/1,1/1,0/1,0/1,1/1,0/1,1/1,1/1,0/1,0/1
63,19,1065145,C,A,,,True,1,0.000418,NaN,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
64,19,1065304,C,A,,,True,1,0.000418,NaN,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
65,19,1065305,G,A,,,True,1,0.000836,NaN,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


## clean up up until here and write tests
## research the QC metrics -> what vars are considered good quality?
## read through the pipeline that ROSMAP used
## add to main.py --> want the all_variants file above concatenated per gene of interest
### prompt: return all the moderate to high coding variants in this set of genes
##### then I can use this package to get the tables for my genes of interest; subset by variants of interest, and add columns that indicate how many of the
### individuals have been sequenced and their APOE4 status... (think about how best to organize that table)

In [74]:
df1 ### add the annotation data to this array 
### save this array and the df2 as a np array
### then use both arrays to subset df2 to get specific variants
### summarize how many of these individuals we have and how many are sequenced alreaady
### save also as csv to make per individual plots

,0,1,2,3,4,5,6
0,19,1040117,A,G,,,True
1,19,1040272,C,T,,,True
2,19,1040308,G,A,,,True
3,19,1040382,G,A,,,True
4,19,1040390,C,T,,,True
...,...,...,...,...,...,...,...
623,19,1065197,G,C,,,True
624,19,1065304,C,A,,,True
625,19,1065305,G,A,,,True
626,19,1065410,G,C,,,True


In [135]:
dd = pd.merge(df1, d[['CHROM', 'POS', 'ID', 'REF', 'ALT', 'EFFECT', 'IMPACT', 'GENE', 'GENEID', 'HGVS_C', 'HGVC_P', 'LOF', 'NMD']], left_on = 1, right_on = 'POS')

In [140]:
dd.loc[np.array(dd['NMD'])== '(ABCA7|ENSG00000064687|1|1.00)']

,0,1,2,3,4,5,6,CHROM,POS,ID,REF,ALT,EFFECT,IMPACT,GENE,GENEID,HGVS_C,HGVC_P,LOF,NMD
11,19,1045035,C,A,,,True,19,1045035,.,C,A,stop_gained,HIGH,ABCA7,ENSG00000064687,c.1250C>A,p.Ser417*,(ABCA7|ENSG00000064687|1|1.00),(ABCA7|ENSG00000064687|1|1.00)
28,19,1053362,G,A,,,True,19,1053362,.,G,A,stop_gained,HIGH,ABCA7,ENSG00000064687,c.3255G>A,p.Trp1085*,(ABCA7|ENSG00000064687|1|1.00),(ABCA7|ENSG00000064687|1|1.00)
33,19,1054255,G,A,,,True,19,1054255,rs201060968,G,A,stop_gained,HIGH,ABCA7,ENSG00000064687,c.3641G>A,p.Trp1214*,(ABCA7|ENSG00000064687|1|1.00),(ABCA7|ENSG00000064687|1|1.00)
55,19,1058685,C,T,,,True,19,1058685,rs146448899,C,T,stop_gained,HIGH,ABCA7,ENSG00000064687,c.5218C>T,p.Gln1740*,(ABCA7|ENSG00000064687|1|1.00),(ABCA7|ENSG00000064687|1|1.00)


In [143]:
np.unique(dd['EFFECT'])

array(['frameshift_variant', 'frameshift_variant&splice_region_variant',
       'missense_variant', 'missense_variant&splice_region_variant',
       'splice_acceptor_variant&intron_variant',
       'splice_donor_variant&intron_variant', 'stop_gained'], dtype=object)

In [95]:
np.unique(dd.loc[:,4])

array(['', '*', 'C'], dtype=object)

In [54]:
pd.concat((df1, df2), axis = 0)

<GenotypeArray shape=(628, 1196, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
1/1 0/1 1/1 0/1 0/1 ... 0/1 1/1 1/1 0/1 0/1

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
“CHROM”, “POS”, “ID”, “REF”, “ALT”, “QUAL”, “FILTER”, “INFO” and “FORMAT”

,0,1,2,3,4,5,6
0,19,1040117,A,G,,,357.480011
1,19,1040272,C,T,,,391.480011
2,19,1040308,G,A,,,5317.850098
3,19,1040382,G,A,,,48615.78125
4,19,1040390,C,T,,,271.480011
...,...,...,...,...,...,...,...
623,19,1065197,G,C,,,921820.125
624,19,1065304,C,A,,,162.479996
625,19,1065305,G,A,,,1239.709961
626,19,1065410,G,C,,,382.480011


In [51]:
np.unique(callset['variants/INFO'])

KeyError: 'variants/INFO'

In [38]:
callset['variants/ALT'][:,2]

array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', 'CTTTTTTT', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', 'TAAA', '', '', '', '', '', '', '', '', '', '',
       'CAAA', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', 'C', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '*', '', '*', '', '*',
       '', '*', '*', '', '', 

In [23]:
callset['variants/REF']

array(['A', 'C', 'G', 'G', 'C', 'C', 'A', 'A', 'A', 'C', 'A', 'A', 'G',
       'G', 'G', 'G', 'A', 'A', 'C', 'G', 'G', 'T', 'A', 'A', 'C', 'G',
       'C', 'T', 'G', 'CCT', 'C', 'G', 'C', 'T', 'AAT', 'G', 'C', 'C',
       'G', 'A', 'C', 'C', 'G', 'G', 'G', 'G', 'T', 'G', 'T', 'C', 'C',
       'A', 'A', 'G', 'C', 'C', 'C', 'C', 'A', 'G', 'G', 'G', 'C', 'C',
       'CT', 'C', 'G', 'G', 'G', 'C', 'C', 'A', 'T', 'C', 'CTTTTT', 'T',
       'G', 'C', 'C', 'C', 'C', 'C', 'A', 'T', 'G', 'C', 'C', 'G', 'C',
       'C', 'G', 'C', 'C', 'G', 'G', 'A', 'C', 'C', 'T', 'C', 'G', 'T',
       'A', 'A', 'T', 'C', 'C', 'C', 'G', 'T', 'G', 'G', 'T', 'A', 'C',
       'G', 'TCCAG', 'C', 'G', 'C', 'C', 'C', 'C', 'T', 'C', 'T', 'TG',
       'A', 'G', 'GCTGCGGGACACCATGCGCGCCATGGGGCTCAGCCGCGCGGTGCT', 'C',
       'A', 'G', 'A', 'C', 'C', 'A', 'C', 'C', 'G', 'G', 'AGGAGCAG', 'A',
       'G', 'C', 'G', 'T', 'G', 'C', 'T', 'C', 'A', 'AGCTGGGCGTGGTGGCAC',
       'G', 'G', 'G', 'TAA', 'A', 'G', 'G', 'T', 'G', 'CA', '

In [216]:
f = open('log.txt', 'w')
callset = allel.read_vcf(path, log = f, fields = '*', region=dictionary['ABCA7'], tabix=path2)
f.close()

In [223]:
sorted(callset.keys())

['variants/1000Gp3_AA_GF',
 'variants/1000Gp3_AF',
 'variants/1000Gp3_HomC',
 'variants/1000Gp3_RA_GF',
 'variants/1000Gp3_RR_GF',
 'variants/AC',
 'variants/ACMG_GENE',
 'variants/ACMG_INHRT',
 'variants/ACMG_MIM_GENE',
 'variants/ACMG_PATH',
 'variants/AF',
 'variants/ALT',
 'variants/AN',
 'variants/ANN',
 'variants/AR_GENE',
 'variants/BaseQRankSum',
 'variants/CADD_phred',
 'variants/CHROM',
 'variants/CLNDBN',
 'variants/CLNDSDB',
 'variants/CLNDSDBID',
 'variants/CLNSIG',
 'variants/COSMIC_CNT',
 'variants/DNA',
 'variants/DP',
 'variants/DS',
 'variants/END',
 'variants/Entrez_gene_id',
 'variants/Essential_gene',
 'variants/ExAC_AF',
 'variants/ExcessHet',
 'variants/FILTER_LowQual',
 'variants/FILTER_PASS',
 'variants/FILTER_VQSRTrancheINDEL99.00to100.00',
 'variants/FILTER_VQSRTrancheINDEL99.00to100.00+',
 'variants/FILTER_VQSRTrancheSNP99.80to100.00',
 'variants/FILTER_VQSRTrancheSNP99.80to100.00+',
 'variants/FS',
 'variants/GDI',
 'variants/GDI-Phred',
 'variants/GERP++_N

In [187]:
import pandas as pd
import numpy as np
gencode = pd.read_table('./human_Release_19_GRCh37p13/gencode.v19.annotation.gff3.gz', comment="#", sep = "\t", names = ['seqname', 'source', 'feature', 'start' , 'end', 'score', 'strand', 'frame', 'attribute'])
gencode_genes = gencode[(gencode.feature == "gene")][['seqname', 'start', 'end', 'attribute']].copy().reset_index().drop('index', axis=1)
gencode_genes['names'] = gencode_genes['attribute'].str.split(';',expand=True).loc[:,5].str.split('=', expand=True).loc[:,1]
gencode_genes['chr'] = [x.replace('chr','') for x in gencode_genes['seqname']]

chromosome_dictionary = dict(zip(gencode_genes['names'], gencode_genes['seqname']))

sele = np.array(gencode_genes[['chr', 'start','end']])
gencode_genes['location'] = [str(x[0]) + ':' + str(x[1]) + '-' + str(x[2]) for x in sele]
position_dictionary = dict(zip(gencode_genes['names'], gencode_genes['location']))

np.save('../human_Release_19_GRCh37p13/chr_dictionary.npy', chromosome_dictionary) 
np.save('./human_Release_19_GRCh37p13/pos_dictionary.npy', position_dictionary) 

FileNotFoundError: [Errno 2] No such file or directory: '../human_Release_19_GRCh37p13/chr_dictionary.npy'

In [188]:
np.save('./human_Release_19_GRCh37p13/pos_dictionary.npy', position_dictionary) 

In [186]:
position_dictionary['ABCA7']

'19:1040102-1065571'